# Module 4 Assignment
### Name: Jaya Marella

In [1]:
#import statements
%matplotlib inline

import pandas as pd
import os
from scipy.sparse import lil_matrix # Needed for building the matrix of user ratings
import scipy.spatial.distance # Needed for calculating pairwise distances
from scipy.spatial.distance import euclidean

In [2]:
#read csv
youtube_df = pd.read_csv('../Module3/data/Youtuber.csv', encoding= 'unicode_escape')
youtube_df

,Rank,Channel Name,Category,Subscribers,Country,Average Views,Average Likes,Average Comments,Content Type
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,T-Series,Music & Dance,258.4M,India,135.2K,5.6K,223,NaN
2,2.0,MrBeast,Video games,236.1M,United States,104M,4M,74K,Humor
3,3.0,Cocomelon - Nursery Rhymes,Education,171.4M,NaN,5.1M,57.1K,0,NaN
4,4.0,SET India,NaN,167.1M,India,27.9K,996,7,NaN
...,...,...,...,...,...,...,...,...,...
1041,996.0,BBC Earth,NaN,12.5M,United States,80.3K,1.7K,79,NaN
1042,997.0,Jamie Nyland,Animation,12.5M,United States,66.7K,2.5K,104,Humor
1043,998.0,janin al chair,NaN,12.5M,Mexico,1.4M,146.1K,485,NaN
1044,999.0,Myke Towers,Music & Dance,12.5M,Colombia,8.3K,485,8,NaN


In [3]:
#change format of column names
old_names = youtube_df.columns
new_names = ['_'.join(c.split()).lower() for c in old_names]
youtube_df.columns = new_names
youtube_df.head()

,rank,channel_name,category,subscribers,country,average_views,average_likes,average_comments,content_type
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,T-Series,Music & Dance,258.4M,India,135.2K,5.6K,223,NaN
2,2.0,MrBeast,Video games,236.1M,United States,104M,4M,74K,Humor
3,3.0,Cocomelon - Nursery Rhymes,Education,171.4M,NaN,5.1M,57.1K,0,NaN
4,4.0,SET India,NaN,167.1M,India,27.9K,996,7,NaN


In [4]:
#drop null values
youtube_df = youtube_df.dropna(how='all')
youtube_df 

,rank,channel_name,category,subscribers,country,average_views,average_likes,average_comments,content_type
1,1.0,T-Series,Music & Dance,258.4M,India,135.2K,5.6K,223,NaN
2,2.0,MrBeast,Video games,236.1M,United States,104M,4M,74K,Humor
3,3.0,Cocomelon - Nursery Rhymes,Education,171.4M,NaN,5.1M,57.1K,0,NaN
4,4.0,SET India,NaN,167.1M,India,27.9K,996,7,NaN
5,5.0,â¿ Kids Diana Show,Animation,118.5M,NaN,5.1M,14.3K,0,Toys
...,...,...,...,...,...,...,...,...,...
1041,996.0,BBC Earth,NaN,12.5M,United States,80.3K,1.7K,79,NaN
1042,997.0,Jamie Nyland,Animation,12.5M,United States,66.7K,2.5K,104,Humor
1043,998.0,janin al chair,NaN,12.5M,Mexico,1.4M,146.1K,485,NaN
1044,999.0,Myke Towers,Music & Dance,12.5M,Colombia,8.3K,485,8,NaN


In [51]:
# change subscribers to numeric

In [5]:
#set index to channel name
youtube_df = youtube_df.set_index(['channel_name'])

In [6]:
#make binary
cluster_df = pd.get_dummies(youtube_df["country"])
cluster_df.head()

,Algeria,Argentina,Bangladesh,Brazil,Colombia,Egypt,El Salvador,France,India,Indonesia,...,Russia,Saudi Arabia,Singapore,South Korea,Spain,Thailand,Turkey,United Kingdom,United States,Vietnam
channel_name,,,,,,,,,,,,,,,,,,,,,
T-Series,False,False,False,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
MrBeast,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
Cocomelon - Nursery Rhymes,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
SET India,False,False,False,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
â¿ Kids Diana Show,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [7]:
cluster_df = cluster_df.apply(lambda x: x.apply(lambda y: 1 if y else 0))

In [8]:
cluster_df.head()

,Algeria,Argentina,Bangladesh,Brazil,Colombia,Egypt,El Salvador,France,India,Indonesia,...,Russia,Saudi Arabia,Singapore,South Korea,Spain,Thailand,Turkey,United Kingdom,United States,Vietnam
channel_name,,,,,,,,,,,,,,,,,,,,,
T-Series,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
MrBeast,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
Cocomelon - Nursery Rhymes,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
SET India,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
â¿ Kids Diana Show,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
cluster_df.columns

Index(['Algeria', 'Argentina', 'Bangladesh', 'Brazil', 'Colombia', 'Egypt',
       'El Salvador', 'France', 'India', 'Indonesia', 'Iran', 'Iraq', 'Italy',
       'Japan', 'Jordan', 'Mexico', 'Morocco', 'Pakistan', 'Peru',
       'Philippines', 'Russia', 'Saudi Arabia', 'Singapore', 'South Korea',
       'Spain', 'Thailand', 'Turkey', 'United Kingdom', 'United States',
       'Vietnam'],
      dtype='object')

In [10]:
from sklearn.cluster import KMeans

In [11]:
k = 7

In [12]:
cluster_model = KMeans(n_clusters=k)

In [37]:
cluster_df.T#.sum(axis=1)

channel_name,T-Series,MrBeast,Cocomelon - Nursery Rhymes,SET India,â¿ Kids Diana Show,Like Nastya,PewDiePie,Vlad and Niki,Zee Music Company,WWE,...,Make Joke Of,Utkarsh Classes,LOUD,Black Eyed Peas,Little Baby Bum en EspaÃ±ol,BBC Earth,Jamie Nyland,janin al chair,Myke Towers,HikakinTV
Algeria,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Argentina,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Bangladesh,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Brazil,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
Colombia,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
Egypt,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
El Salvador,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
France,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
India,1,0,0,1,0,0,0,0,1,0,...,1,1,0,0,0,0,0,0,0,0
Indonesia,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [28]:
cluster_model.fit(cluster_df.T)

/Users/jaya/anaconda3/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


KMeans(n_clusters=7)

In [30]:
cluster_model.inertia_

125.54166666666666

In [31]:
cluster_labels = cluster_model.predict(cluster_df.T)
yt_cluster_df = pd.DataFrame(cluster_labels, index=cluster_df.T.index, columns=["cluster"])

In [32]:
yt_cluster_df["cluster"].value_counts()

cluster
1    24
2     1
5     1
4     1
3     1
6     1
0     1
Name: count, dtype: int64

In [50]:
for cluster_id,group in yt_cluster_df.groupby("cluster"):
    print(cluster_id, group.shape[0])
    display(group.head(5))
    comp_df = pd.DataFrame(
        zip(cluster_df.T.columns, cluster_model.cluster_centers_[cluster_id]),
        columns=["channel", "weight"]
    )
    
    comp_df = comp_df.set_index("channel").join(youtube_df)
    comp_df = comp_df[comp_df["weight"] > 0]
    display(comp_df.sort_values(by="rank", ascending=True).head(5))

0 1


,cluster
United States,0


,weight,rank,category,subscribers,country,average_views,average_likes,average_comments,content_type
channel,,,,,,,,,
MrBeast,1.0,2.0,Video games,236.1M,United States,104M,4M,74K,Humor
PewDiePie,1.0,7.0,Movies,111.6M,United States,1.7M,109.2K,3.1K,Video games
WWE,1.0,10.0,Video games,99M,United States,182.9K,6.8K,227,NaN
HYBE LABELS,1.0,16.0,Music & Dance,73.9M,United States,118.1K,21.6K,215,NaN
Movieclips,1.0,26.0,Movies,60.7M,United States,13.8K,212,11,Humor


1 24


,cluster
Algeria,1
Argentina,1
Bangladesh,1
Egypt,1
El Salvador,1


...

,cluster
Colombia,5


,weight,rank,category,subscribers,country,average_views,average_likes,average_comments,content_type
channel,,,,,,,,,
Daddy Yankee,1.0,88.0,Music & Dance,38.6M,Colombia,56.6K,6.3K,216,NaN
J Balvin,1.0,118.0,Animation,34.2M,Colombia,10.5K,1K,21,Video games
Maluma,1.0,169.0,Music & Dance,29.8M,Colombia,2.2M,66.7K,2.6K,NaN
Ami Rodriguez,1.0,206.0,Animation,27M,Colombia,79.5K,4.7K,278,Humor
NickyJamTV,1.0,276.0,Music & Dance,23.8M,Colombia,15.6K,1.4K,62,Movies


6 1


,cluster
Mexico,6


,weight,rank,category,subscribers,country,average_views,average_likes,average_comments,content_type
channel,,,,,,,,,
JuegaGerman,1.0,43.0,Movies,49.2M,Mexico,1.5M,96.5K,4.8K,Animation
Mikecrack,1.0,46.0,Movies,47.4M,Mexico,785.6K,73.5K,1.7K,Animation
Badabun,1.0,47.0,NaN,47.3M,Mexico,6.8K,276,3,NaN
YOLO AVENTURAS,1.0,60.0,Movies,43.9M,Mexico,2.3M,114.4K,349,Animation
Luisito Comunica,1.0,68.0,NaN,41.8M,Mexico,3.5M,89.8K,1.9K,NaN


In [16]:
for cluster,channels in yt_cluster_df.groupby("cluster"):
    print("Cluster:", cluster, "Size:", channels.shape[0])
    
    top_countries = cluster_df.loc[channels.index].sum().sort_values(ascending=False).head(5) / channels.shape[0]
    print("\t", "Top Countries:")
    for this_c,rate in top_countries.items():
        print("\t\t", this_c, "[%0.4f]" % rate)
    
    #print("\t", "Channel Examples:")
    #for m_id in channels.sample(10).index:
        #print("\t\t", m_id, movie_data_map[m_id]["movie"])

Cluster: 0 Size: 254
	 Top Countries:
		 India [1.0709]
		 Algeria [0.0000]
		 Morocco [0.0000]
		 United States [0.0000]
		 United Kingdom [0.0000]
Cluster: 1 Size: 285
	 Top Countries:
		 United States [1.1123]
		 Algeria [0.0000]
		 Argentina [0.0000]
		 United Kingdom [0.0000]
		 Turkey [0.0000]
Cluster: 2 Size: 312
	 Top Countries:
		 Colombia [0.0641]
		 Thailand [0.0577]
		 Philippines [0.0449]
		 Pakistan [0.0417]
		 Spain [0.0321]
Cluster: 3 Size: 65
	 Top Countries:
		 Brazil [1.1231]
		 Algeria [0.0000]
		 Morocco [0.0000]
		 United States [0.0000]
		 United Kingdom [0.0000]
Cluster: 4 Size: 62
	 Top Countries:
		 Mexico [1.1290]
		 Argentina [0.0000]
		 United States [0.0000]
		 United Kingdom [0.0000]
		 Turkey [0.0000]
Cluster: 5 Size: 40
	 Top Countries:
		 Indonesia [1.0500]
		 Algeria [0.0000]
		 Morocco [0.0000]
		 United States [0.0000]
		 United Kingdom [0.0000]
Cluster: 6 Size: 27
	 Top Countries:
		 Russia [1.1481]
		 Algeria [0.0000]
		 Argentina [0.0000]
		 Unit

In [17]:
#copied this code from github but not sure what the clusters represent or how to find most popular channel in each cluster

In [18]:
k = 5

In [19]:
kmeans_model = KMeans(n_clusters=k, random_state=42)

In [20]:
kmeans_model.fit(cluster_df)

/Users/jaya/anaconda3/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


KMeans(n_clusters=5, random_state=42)

In [21]:
cluster_labels = kmeans_model.labels_

#### steps for cleaning data:
* get rid of null values?
* read each line in df into dictionary 
* turn dictionary into df
* remove spaces from columns
* fill in NaN with unknown in categorical columns
